## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-15-04-06-54 +0000,nyt,Live Updates: Police Say They Plan to Charge S...,https://www.nytimes.com/live/2025/12/14/world/...
1,2025-12-15-04-01-56 +0000,nyt,Here’s how Jimmy Lai became a politically outs...,https://www.nytimes.com/live/2025/12/15/world/...
2,2025-12-15-04-00-00 +0000,wsj,Why Germany Wants a Divorce With China,https://www.wsj.com/world/europe/why-germany-w...
3,2025-12-15-03-55-38 +0000,startribune,Live: Vikings lead the Cowboys 31-23 in the fo...,https://www.startribune.com/minnesota-vikings-...
4,2025-12-15-03-49-00 +0000,nyt,Two Bodies Found at Home Owned by Director Rob...,https://www.nytimes.com/2025/12/14/us/rob-rein...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2390/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
58,bondi,30
59,beach,26
7,shooting,25
81,brown,22
89,trump,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
133,2025-12-14-18-41-53 +0000,nypost,President Trump mourns victims of Bondi Beach ...,https://nypost.com/2025/12/14/us-news/presiden...,150
10,2025-12-15-03-20-03 +0000,wapo,Live updates: 15 killed in Bondi Beach shootin...,https://www.washingtonpost.com/world/2025/12/1...,118
146,2025-12-14-16-36-41 +0000,nypost,"Famed Australian rabbi Eli Schlanger, who just...",https://nypost.com/2025/12/14/world-news/famed...,112
47,2025-12-15-00-36-03 +0000,latimes,Los Angeles Jewish groups ramp up Hanukkah fes...,https://www.latimes.com/california/story/2025-...,109
151,2025-12-14-15-32-12 +0000,nypost,Bondi Beach terror survivor describes how cops...,https://nypost.com/2025/12/14/world-news/bondi...,105


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
133,150,2025-12-14-18-41-53 +0000,nypost,President Trump mourns victims of Bondi Beach ...,https://nypost.com/2025/12/14/us-news/presiden...
146,56,2025-12-14-16-36-41 +0000,nypost,"Famed Australian rabbi Eli Schlanger, who just...",https://nypost.com/2025/12/14/world-news/famed...
156,41,2025-12-14-13-40-25 +0000,cbc,"Australian police suspect father, son behind d...",https://www.cbc.ca/news/world/bondi-beach-shoo...
22,37,2025-12-15-02-47-53 +0000,cbc,"Jimmy Lai, Hong Kong media tycoon and Beijing ...",https://www.cbc.ca/news/world/jimmy-lai-hong-k...
34,31,2025-12-15-01-51-45 +0000,nypost,Mom says viral video shows her being booted fr...,https://nypost.com/2025/12/14/us-news/mom-says...
78,31,2025-12-14-22-16-27 +0000,nypost,Australia’s PM blamed for failing to stand up ...,https://nypost.com/2025/12/14/world-news/austr...
145,29,2025-12-14-17-00-00 +0000,wsj,"For decades, Tonka trucks have been a family s...",https://www.wsj.com/economy/trade/tonka-trucks...
141,26,2025-12-14-17-12-07 +0000,nyt,Chile Poised for Right-Wing Victory as Crime F...,https://www.nytimes.com/2025/12/13/world/ameri...
100,26,2025-12-14-21-19-11 +0000,wapo,She was shot at 15 in a school shooting. Brown...,https://www.washingtonpost.com/nation/2025/12/...
56,25,2025-12-14-23-43-16 +0000,nypost,Holocaust survivors and children among the 15 ...,https://nypost.com/2025/12/14/world-news/holoc...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
